In [99]:
import gurobipy as gp 
from gurobipy import *
import pandas as pd
import numpy as np

In [116]:
#highest benefit first
I = 4
J = 25
K = 15
b = np.array([5,8,12,4,6,6,3,7,6,15,9,5,8,10,1,5,3,7,12,14,5,8,9,8,10])
p = np.array([3,6,7,5,4,2,6,3,5,8,10,2,4,7,1,5,8,3,6,4,12,4,8,4,7])
indexes = np.arange(J)#都是0~24台機器
tup = list(zip(indexes, b))
tup = sorted(tup, key = lambda s: s[1], reverse = True)
bene = np.zeros(I+1) #多一格給底下用
b = [] #製造機器存了誰
accu_p = np.zeros(I)
for i in range(I):
    b.append([])
    #display(b)
display(accu_p)
bene[I] = 1000
display(bene)
for i, j in tup: #先找目前最少 index-1&benefit
    #這步得直接找到人選
    smallest = I #原本第I+1台 不存在的 
    for k in range(0,I):
        if(bene[k]<bene[smallest] and accu_p[k] + p[i] <= K):
            smallest = k
    if(smallest != I):
        bene[smallest]+=j
        b[smallest].append(i+1) #確切的job給誰
        accu_p[smallest] += p[i]
    print(i+1, smallest+1, accu_p, bene[0:I])
display(bene[0:I])
display(b)
display(accu_p)

array([0., 0., 0., 0.])

array([   0.,    0.,    0.,    0., 1000.])

10 1 [8. 0. 0. 0.] [15.  0.  0.  0.]
20 2 [8. 4. 0. 0.] [15. 14.  0.  0.]
3 3 [8. 4. 7. 0.] [15. 14. 12.  0.]
19 4 [8. 4. 7. 6.] [15. 14. 12. 12.]
14 3 [ 8.  4. 14.  6.] [15. 14. 22. 12.]
25 4 [ 8.  4. 14. 13.] [15. 14. 22. 22.]
11 2 [ 8. 14. 14. 13.] [15. 23. 22. 22.]
23 5 [ 8. 14. 14. 13.] [15. 23. 22. 22.]
2 1 [14. 14. 14. 13.] [23. 23. 22. 22.]
13 5 [14. 14. 14. 13.] [23. 23. 22. 22.]
22 5 [14. 14. 14. 13.] [23. 23. 22. 22.]
24 5 [14. 14. 14. 13.] [23. 23. 22. 22.]
8 5 [14. 14. 14. 13.] [23. 23. 22. 22.]
18 5 [14. 14. 14. 13.] [23. 23. 22. 22.]
5 5 [14. 14. 14. 13.] [23. 23. 22. 22.]
6 4 [14. 14. 14. 15.] [23. 23. 22. 28.]
9 5 [14. 14. 14. 15.] [23. 23. 22. 28.]
1 5 [14. 14. 14. 15.] [23. 23. 22. 28.]
12 5 [14. 14. 14. 15.] [23. 23. 22. 28.]
16 5 [14. 14. 14. 15.] [23. 23. 22. 28.]
21 5 [14. 14. 14. 15.] [23. 23. 22. 28.]
4 5 [14. 14. 14. 15.] [23. 23. 22. 28.]
7 5 [14. 14. 14. 15.] [23. 23. 22. 28.]
17 5 [14. 14. 14. 15.] [23. 23. 22. 28.]
15 3 [14. 14. 15. 15.] [23. 23. 23. 28.]


array([23., 23., 23., 28.])

[[10, 2], [20, 11], [3, 14, 15], [19, 25, 6]]

array([14., 14., 15., 15.])

23.0

In [109]:
I = 4
J = 25
K = 15
b = np.array([5,8,12,4,6,6,3,7,6,15,9,5,8,10,1,5,3,7,12,14,5,8,9,8,10])
p = np.array([3,6,7,5,4,2,6,3,5,8,10,2,4,7,1,5,8,3,6,4,12,4,8,4,7])
p1 = Model("p1")
a = []
w = p1.addVar(    lb = 0, vtype = GRB.CONTINUOUS, name = 'w')
a = p1.addVars(I, J, lb = 0, vtype = GRB.CONTINUOUS, name = 'a')
#a = p1.addVars(I, J, lb = 0, vtype = GRB.BINARY, name = 'a') 正常版本
p1.setObjective(w , GRB.MAXIMIZE) 
p1.addConstrs( (w    <= quicksum(a[i, j]*b[j] for j in range(J))for i in range(I)), "w <= aij*bj")
p1.addConstrs((quicksum(a[i, j] for i in range(I)) <= 1 for j in range(J)), "aij")  
p1.addConstrs((quicksum(a[i, j]*p[j] for j in range(J)) <= K for i in range(I)), "aij*pj <= K")
p1.addConstr(  (w    >= 0), "w >= 0" )
p1.addConstrs(    (   (a[i, j] >=0) for i in range(I) for j in range (J)   ), "aij >= 0" )
p1.optimize()

for var in p1.getVars():
    print(var.varName, '= %.2f' %var.x)
print("\n")
print("objective value(max w) = %.2f" %p1.objVal)

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[x86])
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 134 rows, 101 columns and 405 nonzeros
Model fingerprint: 0xcae02e26
Coefficient statistics:
  Matrix range     [1e+00, 2e+01]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 2e+01]
Presolve removed 101 rows and 0 columns
Presolve time: 0.01s
Presolved: 33 rows, 101 columns, 304 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    8.0000000e+30   4.000000e+30   8.000000e+00      0s
      46    3.0392857e+01   0.000000e+00   0.000000e+00      0s

Solved in 46 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.039285714e+01
w = 30.39
a[0,0] = 0.00
a[0,1] = 0.00
a[0,2] = 0.00
a[0,3] = 0.00
a[0,4] = 0.25
a[0,5] = 1.00
a[0,6] = 0.00
a[0,7] = 1.00
a[0,8] = 0.00
a[0,9] = 0.00
a[0,10] = 0.00
a[0,11] = 1.00
a[0,12] = 0.00
a[0,13] = 0.86
a[0,14] 

In [98]:
#範例測資
I = 3
J = 10
K = 15
b = np.array([5, 8, 4, 6, 3, 7, 6, 9, 5, 8])
p = np.array([3, 6, 5, 4, 1, 8, 5, 12,7, 6])
indexes = np.arange(J)#都是0~24台機器
tup = list(zip(indexes, b))
tup = sorted(tup, key = lambda s: s[1], reverse = True)
bene = np.zeros(I+1) #多一格給底下用
b = [] #製造機器存了誰
accu_p = np.zeros(I)
for i in range(I):
    b.append([])
    #display(b)
display(accu_p)
bene[I] = 1000
display(bene)
for i, j in tup: #先找目前最少 index-1&benefit
    #這步得直接找到人選
    smallest = I #原本第I+1台 不存在的 
    for k in range(0,I):
        if(bene[k]<bene[smallest] and accu_p[k] + p[i] <= K):
            smallest = k
    if(smallest != I):
        bene[smallest]+=j
        b[smallest].append(i+1) #確切的job給誰
        accu_p[smallest] += p[i]
    print(i+1, smallest+1, accu_p, bene[0:I])
display(bene[0:I])
display(b)
display(accu_p)

array([0., 0., 0.])

array([   0.,    0.,    0., 1000.])

8 1 [12.  0.  0.] [9. 0. 0.]
2 2 [12.  6.  0.] [9. 8. 0.]
10 3 [12.  6.  6.] [9. 8. 8.]
6 2 [12. 14.  6.] [ 9. 15.  8.]
4 3 [12. 14. 10.] [ 9. 15. 14.]
7 3 [12. 14. 15.] [ 9. 15. 20.]
1 1 [15. 14. 15.] [14. 15. 20.]
9 4 [15. 14. 15.] [14. 15. 20.]
3 4 [15. 14. 15.] [14. 15. 20.]
5 2 [15. 15. 15.] [14. 18. 20.]


array([14., 18., 20.])

[[8, 1], [2, 6, 5], [10, 4, 7]]

array([15., 15., 15.])

In [115]:
#highest ratio first
I = 4
J = 25
K = 15
b = np.array([5,8,12,4,6,6,3,7,6,15,9,5,8,10,1,5,3,7,12,14,5,8,9,8,10])
p = np.array([3,6,7,5,4,2,6,3,5,8,10,2,4,7,1,5,8,3,6,4,12,4,8,4,7])
ratio = b/p
indexes = np.arange(J)#都是0~24台機器
tup = list(zip(indexes, ratio))
tup = sorted(tup, key = lambda s: s[1], reverse = True)
display(tup)
bene = np.zeros(I+1) #多一格給底下用
save = [] #製造機器存了誰
accu_p = np.zeros(I)
for i in range(I):
    save.append([])
    #display(b)
display(accu_p)
bene[I] = 1000
display(bene)
for i, j in tup: #先找目前最少 index-1&benefit
    smallest = I #原本第I+1台 不存在的 
    for k in range(0,I):
        if(bene[k]<bene[smallest] and accu_p[k] + p[i] <= K):
            smallest = k
    if(smallest != I):
        bene[smallest] += b[i]
        save[smallest].append(i+1) #確切的job給誰
        accu_p[smallest] += p[i]
    print(i+1, smallest+1, accu_p, bene[0:I])
display(bene[0:I])
display(save)
display(accu_p)

[(19, 3.5),
 (5, 3.0),
 (11, 2.5),
 (7, 2.3333333333333335),
 (17, 2.3333333333333335),
 (12, 2.0),
 (18, 2.0),
 (21, 2.0),
 (23, 2.0),
 (9, 1.875),
 (2, 1.7142857142857142),
 (0, 1.6666666666666667),
 (4, 1.5),
 (13, 1.4285714285714286),
 (24, 1.4285714285714286),
 (1, 1.3333333333333333),
 (8, 1.2),
 (22, 1.125),
 (14, 1.0),
 (15, 1.0),
 (10, 0.9),
 (3, 0.8),
 (6, 0.5),
 (20, 0.4166666666666667),
 (16, 0.375)]

array([0., 0., 0., 0.])

array([   0.,    0.,    0.,    0., 1000.])

20 1 [4. 0. 0. 0.] [14.  0.  0.  0.]
6 2 [4. 2. 0. 0.] [14.  6.  0.  0.]
12 3 [4. 2. 2. 0.] [14.  6.  5.  0.]
8 4 [4. 2. 2. 3.] [14.  6.  5.  7.]
18 3 [4. 2. 5. 3.] [14.  6. 12.  7.]
13 2 [4. 6. 5. 3.] [14. 14. 12.  7.]
19 4 [4. 6. 5. 9.] [14. 14. 12. 19.]
22 3 [4. 6. 9. 9.] [14. 14. 20. 19.]
24 1 [8. 6. 9. 9.] [22. 14. 20. 19.]
10 2 [ 8. 14.  9.  9.] [22. 29. 20. 19.]
3 1 [15. 14.  9.  9.] [34. 29. 20. 19.]
1 4 [15. 14.  9. 12.] [34. 29. 20. 24.]
5 3 [15. 14. 13. 12.] [34. 29. 26. 24.]
14 5 [15. 14. 13. 12.] [34. 29. 26. 24.]
25 5 [15. 14. 13. 12.] [34. 29. 26. 24.]
2 5 [15. 14. 13. 12.] [34. 29. 26. 24.]
9 5 [15. 14. 13. 12.] [34. 29. 26. 24.]
23 5 [15. 14. 13. 12.] [34. 29. 26. 24.]
15 4 [15. 14. 13. 13.] [34. 29. 26. 25.]
16 5 [15. 14. 13. 13.] [34. 29. 26. 25.]
11 5 [15. 14. 13. 13.] [34. 29. 26. 25.]
4 5 [15. 14. 13. 13.] [34. 29. 26. 25.]
7 5 [15. 14. 13. 13.] [34. 29. 26. 25.]
21 5 [15. 14. 13. 13.] [34. 29. 26. 25.]
17 5 [15. 14. 13. 13.] [34. 29. 26. 25.]


array([34., 29., 26., 25.])

[[20, 24, 3], [6, 13, 10], [12, 18, 22, 5], [8, 19, 1, 15]]

array([15., 14., 13., 13.])